## This notebook is an example of how to pipeline two models. 
This notebook is an example how to use DeGirum PySDK to do AI inference of a graphical file using two AI models: person detection and pose detection. The person detection model is run on the image and the results are then processed by the pose detection model, one person bbox at a time.
Combined result is then displayed.
This script uses PIL as image processing backend.


In [1]:
import degirum as dg # import DeGirum PySDK
import mytools

### Specify inference option here

In [ ]:
# Please uncomment and edit one of the following inference options to specify your system configuration case according to
# https://cs.degirum.com/doc/0.5.0/degirum.html#system-configuration-for-specific-use-cases

# 1. DeGirum Cloud Zoo inference:
#zoo = dg.connect_model_zoo("dgcps://cs.degirum.com", token=mytools.token_get())

# 2. AIServer inference via IP address using models from DeGirum Cloud model zoo
#zoo = dg.connect_model_zoo(("192.168.0.7", "https://cs.degirum.com/degirum_com/public"), token=mytools.token_get())

# 3. AIServer inference via IP address using local model zoo
#zoo = dg.connect_model_zoo("192.168.0.1")

# 4. ORCA board installed locally using models from DeGirum Cloud Model Zoo
#zoo = dg.connect_model_zoo("https://cs.degirum.com/degirum_com/public", token=mytools.token_get())

# 5. Local inference with locally deployed model
#zoo = dg.connect_model_zoo("full/path/to/model.json")

In [ ]:
# load models for DeGirum Orca AI accelerator
# (change model name to "...n2x_cpu_1" to run it on CPU)
people_det_model = zoo.load_model("yolo_v5s_person_det--512x512_quant_n2x_orca_1")
pose_model = zoo.load_model("mobilenet_v1_posenet_coco_keypoints--353x481_quant_n2x_orca_1")

# adjust some model properties
pose_model.overlay_show_labels = False
pose_model.overlay_line_width = 1
pose_model.overlay_alpha = 1
pose_model._model_parameters.PoseThreshold = 0.5

In [ ]:
# detect persons
people = people_det_model('./images/ThreePersons.jpg')

# for each detected person detect poses
with pose_model: # performance optimization to keep connection to pose_model open
    for person in people.results:
        person_box = people.image.crop(person['bbox'])
        poses = pose_model(person_box)
        display(poses.image_overlay)

In [ ]:
# alternatively, detect all poses
all_poses = pose_model('./images/ThreePersons.jpg')
display(all_poses.image_overlay)